In [2]:
# Function to calculate LULUCF fluxes and carbon densities
# Operates pixel by pixel, so uses numba (Python compiled to C++).
@jit(nopython=True)
def LULUCF_fluxes(in_dict_uint8, in_dict_int16, in_dict_float32):

    # Separate dictionaries for output numpy arrays of each datatype, named by output type (e.g., AGC density, BGC removals) and year ([output_type}_[year]).
    # This is because a dictionary in a Numba function cannot have arrays with multiple data types, so each dictionary has to store only one data type,
    # just like inputs to the function.
    out_dict_uint32 = {}
    out_dict_float32 = {}

    end_years = list(range(first_year, last_year+1, interval_years))[1:]
    # end_years = [2005, 2010]

    agc_dens_curr_block = in_dict_float32[agc_2000]
    bgc_dens_curr_block = in_dict_float32[bgc_2000]
    deadwood_c_dens_curr_block = in_dict_float32[deadwood_c_2000]
    litter_c_dens_curr_block = in_dict_float32[litter_c_2000]
    soil_c_dens_curr_block = in_dict_int16[soil_c_2000]

    r_s_ratio_block = in_dict_float32[r_s_ratio]

    for year in end_years:

        # print(year)

        # Creates array for each input
        LC_prev_block = in_dict_uint8[f"{land_cover}_{year-interval_years}"]
        LC_curr_block = in_dict_uint8[f"{land_cover}_{year}"]
        veg_h_prev_block = in_dict_uint8[f"{vegetation_height}_{year-interval_years}"]
        veg_h_curr_block = in_dict_uint8[f"{vegetation_height}_{year}"]
        planted_forest_type_block = in_dict_uint8[planted_forest_type_layer]
        planted_forest_tree_crop_block = in_dict_uint8[planted_forest_tree_crop_layer]

        burned_area_t_4_block = in_dict_uint8[f"{burned_area}_{year-4}"]
        burned_area_t_3_block = in_dict_uint8[f"{burned_area}_{year-3}"]
        burned_area_t_2_block = in_dict_uint8[f"{burned_area}_{year-2}"]
        burned_area_t_1_block = in_dict_uint8[f"{burned_area}_{year-1}"]
        burned_area_t_block = in_dict_uint8[f"{burned_area}_{year}"]

        forest_dist_t_4_block = in_dict_uint8[f"{forest_disturbance}_{year-4}"]
        forest_dist_t_3_block = in_dict_uint8[f"{forest_disturbance}_{year-3}"]
        forest_dist_t_2_block = in_dict_uint8[f"{forest_disturbance}_{year-2}"]
        forest_dist_t_1_block = in_dict_uint8[f"{forest_disturbance}_{year-1}"]
        forest_dist_t_block = in_dict_uint8[f"{forest_disturbance}_{year}"]

        # Numpy arrays for outputs that don't depend on previous values
        state_out = np.zeros(in_dict_float32[agc_2000].shape).astype('uint32') 
        agc_flux_out_block = np.zeros(in_dict_float32[agc_2000].shape).astype('float32')
        bgc_flux_out_block = np.zeros(in_dict_float32[agc_2000].shape).astype('float32')

        
        # Iterates through all pixels in the chunk
        for row in range(LC_curr_block.shape[0]):
            for col in range(LC_curr_block.shape[1]):
                
                LC_prev = LC_prev_block[row, col]
                LC_curr = LC_curr_block[row, col]
                veg_h_prev = veg_h_prev_block[row, col]
                veg_h_curr = veg_h_curr_block[row, col]
                planted_forest_type = planted_forest_type_block[row, col]
                planted_forest_tree_crop = planted_forest_tree_crop_block[row, col]

                # Note: Stacking the burned area rasters using ndstack outside the pixel iteration did not work with numba.
                # So just reading each burned area raster separately.
                burned_area_t_4 = burned_area_t_4_block[row, col]
                burned_area_t_3 = burned_area_t_3_block[row, col]
                burned_area_t_2 = burned_area_t_2_block[row, col]
                burned_area_t_1 = burned_area_t_1_block[row, col]
                burned_area_t = burned_area_t_block[row, col]
                burned_area_last = max([burned_area_t_4, burned_area_t_3, burned_area_t_2, burned_area_t_1, burned_area_t])  # Most recent year with burned area during the interval

                forest_dist_t_4 = forest_dist_t_4_block[row, col]
                forest_dist_t_3 = forest_dist_t_3_block[row, col]
                forest_dist_t_2 = forest_dist_t_2_block[row, col]
                forest_dist_t_1 = forest_dist_t_1_block[row, col]
                forest_dist_t = forest_dist_t_block[row, col]
                forest_dist_last = max([forest_dist_t_4, forest_dist_t_3, forest_dist_t_2, forest_dist_t_1, forest_dist_t])  # Most recent year with forest disturbance during the interval   

                agc_dens_curr = agc_dens_curr_block[row, col]
                bgc_dens_curr = bgc_dens_curr_block[row, col]
                deadwood_c_dens_curr = deadwood_c_dens_curr_block[row, col]
                litter_c_dens_curr = litter_c_dens_curr_block[row, col]
                soil_c_dens_curr = soil_c_dens_curr_block[row, col]

                r_s_ratio_cell = r_s_ratio_block[row, col]

                tree_prev = (veg_h_prev >= tree_threshold)
                tree_curr = (veg_h_curr >=  tree_threshold)
                tall_veg_prev = (((LC_prev >= tree_dry_min_height_code) and (LC_prev <= tree_dry_max_height_code)) or
                                        ((LC_prev >= tree_wet_min_height_code) and (LC_prev <= tree_wet_max_height_code)))
                tall_veg_curr = (((LC_curr >= tree_dry_min_height_code) and (LC_curr <= tree_dry_max_height_code)) or
                                       ((LC_curr >= tree_wet_min_height_code) and (LC_curr <= tree_wet_max_height_code)))
                short_med_veg_prev = (((LC_prev >= 2) and (LC_prev <= 26)) or
                                       ((LC_prev >= 102) and (LC_prev <= 126)))
                short_med_veg_curr = (((LC_curr >= 2) and (LC_curr <= 26)) or
                                       ((LC_curr >= 102) and (LC_curr <= 126)))

                sig_height_loss_prev_curr = (veg_h_prev-veg_h_curr >= sig_height_loss_threshold) 

                node = 0
                
                ### Tree gain
                if (not tree_prev) and (tree_curr):                  # Non-tree converted to tree (1)    ##TODO: Include mangrove exception.
                    node = accrete_node(node, 1)
                    if planted_forest_type == 0:                     # New non-SDPT trees (11)
                        node = accrete_node(node, 1)
                        if not tall_veg_curr:                        # New trees outside forests (111)
                            node = accrete_node(node, 1)
                            state_out[row, col] = node
                            agc_rf = 2.8
                            agc_flux_out_block[row, col] = (agc_rf*interval_years)*-1
                            agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]   
                            bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                            bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col]   
                        else:                                        # New terrestrial natural forest (112)
                            node = accrete_node(node, 2)
                            state_out[row, col] = node
                            agc_rf = 5.6
                            agc_flux_out_block[row, col] = (agc_rf*interval_years)*-1
                            agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                            bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                            bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                    else:                                            # New SDPT trees (12)
                        node = accrete_node(node, 2)
                        state_out[row, col] = node
                        agc_rf = 10
                        agc_flux_out_block[row, col] = (agc_rf*interval_years)*-1
                        agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]   
                        bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                        bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                
                ### Tree loss
                elif (tree_prev) and (not tree_curr):                # Tree converted to non-tree (2)    ##TODO: Include forest disturbance condition.  ##TODO: Include mangrove exception.
                    node = 2
                    if planted_forest_type == 0:                     # Full loss of non-SDPT trees (21)
                        node = accrete_node(node, 1)
                        if not tall_veg_prev:                        # Full loss of trees outside forests (211)
                            node = accrete_node(node, 1)
                            if burned_area_last == 0:                # Full loss of trees outside forests without fire (2111)
                                node = accrete_node(node, 1)
                                state_out[row, col] = node
                                agc_ef = 0.8
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of trees outside forests with fire (2112)
                                node = accrete_node(node, 2)
                                state_out[row, col] = node
                                agc_ef = 0.6
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                        else:                                        # Full loss of natural forest (212)
                            node = accrete_node(node, 2)
                            if LC_curr == cropland:                  # Full loss of natural forest converted to cropland (2121)
                                node = accrete_node(node, 1)
                                if burned_area_last == 0:            # Full loss of natural forest converted to cropland, not burned (21211)
                                    node = accrete_node(node, 1)
                                    state_out[row, col] = node
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of natural forest converted to cropland, burned (21212)
                                    node = accrete_node(node, 2)
                                    state_out[row, col] = node
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            elif short_med_veg_curr:                 # Full loss of natural forest converted to short or medium vegetation (2122)
                                node = accrete_node(node, 2)
                                if burned_area_last == 0:            # Full loss of natural forest converted to short or medium vegetation, not burned (21221)
                                    node = accrete_node(node, 1)
                                    state_out[row, col] = node
                                    agc_ef = 0.9
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of natural forest converted to short or medium vegetation, burned (21222)
                                    node = accrete_node(node, 2)
                                    state_out[row, col] = node
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            elif LC_curr == builtup:                 # Full loss of natural forest converted to builtup (2123)
                                node = accrete_node(node, 3)
                                if burned_area_last == 0:            # Full loss of natural forest converted to builtup, not burned (21231)
                                    node = accrete_node(node, 1)
                                    state_out[row, col] = node
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of natural forest converted to builtup, burned (21232)
                                    node = accrete_node(node, 2)
                                    state_out[row, col] = node
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of natural forest converted to anything else (2124)
                                node = accrete_node(node, 4)
                                if burned_area_last == 0:            # Full loss of natural forest converted to anything else, not burned (21241)
                                    node = accrete_node(node, 1)
                                    state_out[row, col] = node
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of natural forest converted to anything else, burned (21242)
                                    node = accrete_node(node, 2)
                                    state_out[row, col] = node       
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                    else:                                            # Full loss of SDPT trees (22)
                        node = accrete_node(node, 2)
                        if LC_curr == cropland:                      # Full loss of SDPT converted to cropland (221)
                            node = accrete_node(node, 1)
                            if burned_area_last == 0:                # Full loss of SDPT converted to cropland, not burned (2211)
                                node = accrete_node(node, 1)
                                state_out[row, col] = node
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of SPDPT converted to cropland, burned (2212)
                                node = accrete_node(node, 2)
                                state_out[row, col] = node
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                        elif short_med_veg_curr:                     # Full loss of SDPT converted to short or medium vegetation (222)
                            node = accrete_node(node, 2)
                            if burned_area_last == 0:                # Full loss of SDPT converted to short or medium vegetation, not burned (2221)
                                node = accrete_node(node, 1)
                                state_out[row, col] = node
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of SDPT converted to short or medium vegetation, burned (2222)
                                node = accrete_node(node, 2)
                                state_out[row, col] = node
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                        elif LC_curr == builtup:                     # Full loss of SDPT converted to builtup (223)
                            node = accrete_node(node, 3)
                            if planted_forest_tree_crop == 1:        # Full loss of SDPT planted forest to builtup (2231)
                                node = accrete_node(node, 1)
                                if burned_area_last == 0:            # Full loss of SDPT planted forest converted to builtup, not burned (22311)
                                    node = accrete_node(node, 1)
                                    state_out[row, col] = node
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of SDPT planted forest converted to builtup, burned (22312)
                                    node = accrete_node(node, 2)
                                    state_out[row, col] = node
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of SDPT tree crop to builtup (2232)
                                node = accrete_node(node, 2)
                                if burned_area_last == 0:            # Full loss of SDPT tree crop converted to builtup, not burned (22321)
                                    node = accrete_node(node, 1)
                                    state_out[row, col] = node
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                                else:                                # Full loss of SDPT tree crop converted to builtup, burned (22322)
                                    node = accrete_node(node, 2)
                                    state_out[row, col] = node         
                                    agc_ef = 0.3
                                    agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                    agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                    bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                    bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                        else:                                        # Full loss of SDPT converted to anything else (224)
                            node = accrete_node(node, 4)
                            if burned_area_last == 0:                # Full loss of SDPT converted to builtup, not burned (2241)
                                node = accrete_node(node, 1)
                                state_out[row, col] = node
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                            else:                                    # Full loss of SDPT converted to builtup, burned (2242)
                                node = accrete_node(node, 2)
                                state_out[row, col] = node
                                agc_ef = 0.3
                                agc_flux_out_block[row, col] = (agc_dens_curr * agc_ef)
                                agc_dens_curr_block[row, col] = agc_dens_curr - agc_flux_out_block[row, col]
                                bgc_flux_out_block[row, col] = float(agc_flux_out_block[row, col]) * r_s_ratio_cell
                                bgc_dens_curr_block[row, col] = bgc_dens_curr - bgc_flux_out_block[row, col] 
                
                # ### Trees remaining trees
                # elif (tree_prev) and (tree_curr):                    # Trees remaining trees (3)    ##TODO: Include mangrove exception.
                #     node = accrete_node(node, 3)
                #     if forest_dist_last == 0:                        # Trees without stand-replacing disturbances in the last interval (31)
                #         node = accrete_node(node, 1)
                #         if planted_forest_type == 0:                 # Non-planted trees without stand-replacing disturbance in the last interval (311)
                #             node = accrete_node(node, 1)
                #             if not tall_veg_curr:                    # Trees outside forests without stand-replacing disturbance in the last interval (3111)
                #                 node = accrete_node(node, 1)
                #                 if not sig_height_loss_prev_curr:    # Stable trees outside forests (31111)
                #                     node = accrete_node(node, 1)
                #                     state_out[row, col] = node
                #                 else:                                # Partially disturbed trees outside forests (31112)
                #                     node = accrete_node(node, 2)
                #                     if burned_area_last == 0:        # Partially disturbed trees outside forests without fire (311121)
                #                         node = accrete_node(node, 1)
                #                         state_out[row, col] = node
                #                     else:
                #                         node = accrete_node(node, 2)
                #                         state_out[row, col] = node
                #             else:                                    # Natural forest without stand-replacing disturbance in the last interval (3112)
                #                 node = accrete_node(node, 2)
                #                 state_out[row, col] = node
                #                 # if not sig_height_loss_prev_curr:    # Stable natural forest (31121)
                                    
                                
                            
                #     else:
                #         state_out[row, col] = 32
                    
                else:                                                # Not covered in above branches
                    state_out[row, col] = 4000000000                 # High value for uint32
        
        # Adds the output arrays to the dictionary with the appropriate data type
        # Outputs need .copy() so that previous intervals' arrays in dicationary aren't overwritten because arrays in dictionaries are mutable (courtesy of ChatGPT).
        year_range = f"{year-interval_years}_{year}"
        out_dict_uint32[f"{land_state_pattern}_{year_range}"] = state_out.copy()  
        out_dict_float32[f"{agc_dens_pattern}_{year_range}"] = agc_dens_curr_block.copy()
        out_dict_float32[f"{bgc_dens_pattern}_{year_range}"] = bgc_dens_curr_block.copy()
        out_dict_float32[f"{agc_flux_pattern}_{year_range}"] = agc_flux_out_block.copy()
        out_dict_float32[f"{bgc_flux_pattern}_{year_range}"] = bgc_flux_out_block.copy()

    return out_dict_uint32, out_dict_float32

NameError: name 'jit' is not defined

In [ ]:
%%time

## Create LULUCF flux and carbon stock 2x2 deg rasters 

## Area to analyze
## chunk_params arguments: W, S, E, N, chunk size (degrees)
# chunk_params = [-180, -60, 180, 80, 2]  # entire world
# chunk_params = [-10, 40, 20, 70, 1]    # 30x30 deg (70N_010W), 900 chunks

# chunk_params = [-10, 60, 0, 70, 1]    # 10x10 deg (70N_010W), 100 chunks
# chunk_params = [-10, 65, -5, 70, 1]    # 5x5 deg (70N_010W), 25 chunks
# chunk_params = [-10, 68, -8, 70, 1]    # 2x2 deg (70N_010W), 4 chunks
# chunk_params = [-10, 69, -9, 70, 1]    # 1x1 deg (70N_010W), 1 chunk

# chunk_params = [10, 40, 20, 50, 2]    # 10x10 deg (50N_010E), 25 chunks
# chunk_params = [10, 40, 20, 50, 10]    # 10x10 deg (50N_010E), 1 chunk
# chunk_params = [10, 46, 14, 50, 2]   # 4x4 deg, 4 chunks
# chunk_params = [110, -10, 114, -6, 2]   # 4x4 deg, 4 chunks
# chunk_params = [10, 48, 12, 50, 1]   # 2x2 deg, 4 chunks
# chunk_params = [10, 49, 11, 50, 1]   # 1x1 deg, 1 chunk
# chunk_params = [10, 49, 11, 50, 0.5] # 1x1 deg, 4 chunks
# chunk_params = [10, 49.5, 10.5, 50, 0.25] # 0.5x0.5 deg, 4 chunks
# chunk_params = [10, 42, 11, 43, 0.5] # 1x1 deg, 4 chunks (some GLCLU code=254 for ocean and some land, so data should be output)
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data, no fire)
chunk_params = [15, 41.75, 15.25, 42, 0.25] # 0.25x0.25 deg, 1 chunk (has data with fire)

# # Range of no-data cases for testing
# chunk_params = [20, 69.75, 20.25, 70, 0.25] # 0.25x0.25 deg, 1 chunk (tile exists for GLCLU but not all other inputs, e.g., fire)
# chunk_params = [110, -10, 120, 0, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [110, -20, 120, -10, 2]    # 10x10 deg (00N_110E), 25 chunks (all chunks have land and should be output)
# chunk_params = [0, 79.75, 0.25, 80, 0.25] # 0.25x0.25 deg, 1 chunk (no 80N_000E tile-- no data)
# chunk_params = [112, -12, 116, -8, 2]   # 2x2 deg, 1 chunk (bottom of Java, has data but mostly ocean)
# chunk_params = [10.875, 41.75, 11, 42, 0.25] # 0.25x0.25 deg, 1 chunk (entirely GLCLU code=255 for ocean, so no actual data-- nothing should be be output)
# chunk_params = [-10, 21.75, -9.75, 22, 0.25] # 0.25x0.25 deg, 1 chunk (has data but entirely desert (fully GLCLU code=0))
# chunk_params = [10, 49.75, 10.25, 50, 0.25] # 0.25x0.25 deg, 1 chunk (has data)


# Makes list of chunks to analyze
chunks = get_chunk_bounds(chunk_params)  
print("Processing", len(chunks), "chunks")
# print(chunks)

# Determines if the output file names for final versions of outputs should be used
is_final = False
if len(chunks) > 90:
    is_final = True
    print("Running as final model.")

# Creates list of tasks to run (1 task = 1 chunk for all years)
delayed_result = [dask.delayed(calculate_and_upload_LULUCF_fluxes)(chunk, is_final) for chunk in chunks]

# Actually runs analysis
results = dask.compute(*delayed_result)
results

In [ ]:
# Execute without Dask
calculate_and_upload_LULUCF_fluxes([10, 49.75, 10.25, 50], is_final=False)

In [ ]:
def get_tile_dataset_rio(uri, bounds, chunk_length_pixels, no_data_val):

    bounds_str = boundstr(bounds)

    try:
        print("trying")
        with rasterio.open(uri) as ds:
            window = rasterio.windows.from_bounds(*bounds, ds.transform)
            data = ds.read(1, window=window)
    except:
        print("excepting")
        data = np.full((chunk_length_pixels, chunk_length_pixels), no_data_val)

    return data

get_tile_dataset_rio("s3://gfw2-data/climate/carbon_model/other_emissions_inputs/burn_year/burn_year_10x10_clip/ba_2017_70N_020E.tif", [20, 69.75, 20.25, 70], 1000, 255)